In [2]:
import lightgbm
import pandas as pd
import numpy as np

In [3]:
import os
import codecs
import math

import sklearn.metrics
import sklearn
import numpy


In [4]:
!pwd

/Users/andyzt/ClionProjects/myGboosting/testing


In [239]:
df_train = pd.read_csv("datasets/FacebookCommentVolumeDataset/train.csv", header=None)

In [240]:
y_train = df_train.iloc[:,53]
X_train = df_train.iloc[:,:53]
X_train.shape

(40949, 53)

In [241]:
df_test = pd.read_csv("datasets/FacebookCommentVolumeDataset/test.csv", header=None)

In [242]:
y_test = df_test.iloc[:,53]
X_test = df_test.iloc[:,:53]
X_test.shape

(10044, 53)

## LightGBM

In [267]:
D_train = lightgbm.Dataset(X_train, y_train)


lgb_params = dict()
lgb_params['boosting_type'] = 'gbdt'  # space['boosting_type'], # 'gbdt', # gbdt | dart | goss
# px['objective'] ='multi:softprob'
lgb_params['application'] = 'regression'
lgb_params['metric'] = 'l2'
lgb_params['learning_rate'] = 1.0
#lgb_params['num_leaves'] = 100
lgb_params['min_data_in_leaf'] = 10
#lgb_params['min_sum_hessian_in_leaf'] = 1e-3
lgb_params['max_depth'] = 2
lgb_params['lambda_l1'] = 0.0  # space['lambda_l1'],
lgb_params['lambda_l2'] = 0.0  # space['lambda_l2'],
lgb_params['max_bin'] = 10
lgb_params['feature_fraction'] = 1.0
lgb_params['bagging_fraction'] = 0.66
lgb_params['bagging_freq'] = 1
lgb_params['num_threads'] = 1



model = lightgbm.train(lgb_params,
                       D_train,
                       num_boost_round=2,
                       # metrics='mlogloss',
                       #valid_sets=D_val,
                       # valid_names='val',
                       # fobj=None,
                       # feval=None,
                       # init_model=None,
                       # feature_name='auto',
                       # categorical_feature='auto',
                       #early_stopping_rounds=100,
                       # evals_result=None,
                       verbose_eval=False,
                       # learning_rates=None,
                       # keep_training_booster=False,
                       # callbacks=None
                       )

#nb_trees = model.best_iteration

#print('nb_trees={}'.format(nb_trees))

#y_pred = model.predict(X_test, num_iteration=nb_trees )
#test_loss = sklearn.metrics.log_loss( y_test, y_pred, labels=list(range(10)))

In [268]:
y_pred = model.predict(X_train)
train_loss = sklearn.metrics.mean_squared_error(y_train, y_pred)

#acc = sklearn.metrics.accuracy_score( y_train, numpy.argmax( y_pred, axis=1 ) )

print('train_loss={}'.format(train_loss))

train_loss=852.4929342275067


In [99]:
y_pred = model.predict(X_test)
test_loss = sklearn.metrics.mean_squared_error(y_test, y_pred)

#acc = sklearn.metrics.accuracy_score( y_train, numpy.argmax( y_pred, axis=1 ) )

print('test_loss={}'.format(test_loss))

test_loss=10976.949481412941


## XGBoost

In [100]:
import xgboost

In [101]:
D_train = xgboost.DMatrix(X_train, y_train)
D_test = xgboost.DMatrix(X_test)

xgb_params = dict()
xgb_params['eta'] = 1.0
xgb_params['max_depth'] = 5
xgb_params['subsample'] = 0.66
xgb_params['min_child_weight'] = 10
#xgb_params['gamma'] = 0.271712091643
xgb_params['colsample_bytree'] = 1.0
#xgb_params['colsample_bylevel'] = 0.79407015729
xgb_params['objective'] = 'reg:linear'
#xgb_params['eval_metric'] = [ 'merror', 'mlogloss']
#xgb_params['num_class'] = 10
xgb_params['seed'] = 123456
xgb_params['silent'] = False
xgb_params['tree_method'] = 'hist'
xgb_params['max_bin'] = 10
xgb_params['nthread'] = 1


#xgb_params['updater'] = 'grow_gpu'


#watchlist = [(D_train, 'train'), (D_val, 'valid')]

model = xgboost.train( params=xgb_params,
                      dtrain=D_train,
                      num_boost_round=20,
                      #evals=watchlist,
                      #verbose_eval=10,
                      #early_stopping_rounds=50
                     )

print('nb_trees={}'.format(model.best_ntree_limit))



nb_trees=20


In [103]:
y_pred = model.predict(D_train)
train_loss = sklearn.metrics.mean_squared_error(y_train, y_pred)

#acc = sklearn.metrics.accuracy_score( y_train, numpy.argmax( y_pred, axis=1 ) )

print('train_loss={}'.format(train_loss))

train_loss=466.8945376161178


In [104]:
y_pred = model.predict(D_test)

test_loss = sklearn.metrics.mean_squared_error(y_test, y_pred)

print('test_loss={}'.format(test_loss))

test_loss=11647.723753997956


расчет MSE по предсказаниям нашей модели

In [65]:
!pwd

/Users/andyzt/ClionProjects/myGboosting/testing


In [105]:
df_pred_train = pd.read_csv("../cmake-build-release/pred_train.csv", header=None)

In [106]:
train_loss = sklearn.metrics.mean_squared_error(y_train, df_pred_train)

print('train_loss={}'.format(train_loss))

train_loss=641.3193536717484


In [47]:
df_pred = pd.read_csv("../cmake-build-release/pred.csv", header=None)

In [48]:
test_loss = sklearn.metrics.mean_squared_error(y_test, df_pred)

print('test_loss={}'.format(test_loss))

test_loss=1.4471964093797176


## Higgs dataset testing (LightGBM)

In [5]:
df = pd.read_csv("datasets/Higgs/train_orig.csv.gz")

In [6]:
#df_train['target'] = np.where(df_train['Label'] == 's' ,1,0)

In [7]:
del df['Label']
y = df['Weight']
del df['Weight']

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)

In [38]:
D_train = lightgbm.Dataset(X_train, y_train)


lgb_params = dict()
lgb_params['boosting_type'] = 'gbdt'  # space['boosting_type'], # 'gbdt', # gbdt | dart | goss
# px['objective'] ='multi:softprob'
lgb_params['application'] = 'regression'
lgb_params['metric'] = 'l2'
lgb_params['learning_rate'] = 0.5
lgb_params['num_leaves'] = 64
#lgb_params['min_data_in_leaf'] = 10
#lgb_params['min_sum_hessian_in_leaf'] = 1e-3
#lgb_params['max_depth'] = 6
lgb_params['lambda_l1'] = 0.0  # space['lambda_l1'],
lgb_params['lambda_l2'] = 0.0  # space['lambda_l2'],
lgb_params['max_bin'] = 255
lgb_params['feature_fraction'] = 1.0
lgb_params['bagging_fraction'] = 0.5
lgb_params['bagging_freq'] = 1
lgb_params['num_threads'] = 1



model = lightgbm.train(lgb_params,
                       D_train,
                       num_boost_round=400,
                       # metrics='mlogloss',
                       #valid_sets=D_val,
                       # valid_names='val',
                       # fobj=None,
                       # feval=None,
                       # init_model=None,
                       # feature_name='auto',
                       # categorical_feature='auto',
                       #early_stopping_rounds=100,
                       # evals_result=None,
                       verbose_eval=False,
                       # learning_rates=None,
                       # keep_training_booster=False,
                       # callbacks=None
                       )

In [39]:
y_pred = model.predict(X_train)
train_loss = sklearn.metrics.mean_squared_error(y_train, y_pred)

#acc = sklearn.metrics.accuracy_score( y_train, numpy.argmax( y_pred, axis=1 ) )

print('train_loss={}'.format(train_loss))

train_loss=0.6468995535097333


In [40]:
y_pred = model.predict(X_test)
test_loss = sklearn.metrics.mean_squared_error(y_test, y_pred)

#acc = sklearn.metrics.accuracy_score( y_train, numpy.argmax( y_pred, axis=1 ) )

print('test_loss={}'.format(test_loss))

test_loss=1.8866258994568623


In [202]:
df_train = X_train
df_train['target'] = y_train

/Users/andyzt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [205]:
df_train.to_csv("train.csv",index=False)

In [206]:
df_test = X_test
df_test['target'] = y_test
df_test.to_csv("test.csv",index=False)

/Users/andyzt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## XGBoost on Higgs

In [13]:
import xgboost

In [17]:
%%time
D_train = xgboost.DMatrix(X_train, y_train)
D_test = xgboost.DMatrix(X_test)

xgb_params = dict()
xgb_params['eta'] = 1.0
xgb_params['max_depth'] = 6
xgb_params['subsample'] = 1.0
xgb_params['min_child_weight'] = 1
#xgb_params['gamma'] = 0.271712091643
xgb_params['colsample_bytree'] = 1.0
#xgb_params['colsample_bylevel'] = 0.79407015729
xgb_params['objective'] = 'reg:linear'
#xgb_params['eval_metric'] = [ 'merror', 'mlogloss']
#xgb_params['num_class'] = 10
xgb_params['seed'] = 123456
xgb_params['silent'] = False
xgb_params['tree_method'] = 'hist'
xgb_params['max_bin'] = 10
xgb_params['nthread'] = 1


#xgb_params['updater'] = 'grow_gpu'


#watchlist = [(D_train, 'train'), (D_val, 'valid')]

model = xgboost.train( params=xgb_params,
                      dtrain=D_train,
                      num_boost_round=400,
                      #evals=watchlist,
                      #verbose_eval=10,
                      #early_stopping_rounds=50
                     )

print('nb_trees={}'.format(model.best_ntree_limit))




nb_trees=400
CPU times: user 15.9 s, sys: 143 ms, total: 16 s
Wall time: 16.1 s


In [15]:
y_pred = model.predict(D_train)
train_loss = sklearn.metrics.mean_squared_error(y_train, y_pred)

#acc = sklearn.metrics.accuracy_score( y_train, numpy.argmax( y_pred, axis=1 ) )

print('train_loss={}'.format(train_loss))

train_loss=0.7874981669733602
